## Build an emission model

This notebook demonstrates how to prepare **DELWAQ D-Emissions** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

In HydroMT, you can interact with **DELWAQ** models either to be used by **D-Emissions (demission)** or **D-Water Quality (delwaq)**.

Lets first check if the delwaq and demission models are recognized by hydromt

In [ ]:
# this should return "delwaq, demission, wflow, wflow_sediment"
!hydromt --models

Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [ ]:
!hydromt build --help

### Setup demission model base layers 

DELWAQ models are quite specific in the sense that they are usually added on top of an already existing hydrologic / hydraulic or hydrodynamic model.
In HydroMT, for now, DELWAQ models (demission for D-Emissions and delwaq for D-Water Quality) can only be built on top of [Wflow](https://deltares.github.io/Wflow.jl/stable/) hydrologic models. You can find more information on this coupling in [docs(coupling_wflow)](https://deltares.github.io/hydromt_delwaq/latest/advanced/coupling_wflow.html).

Here is how you can build a D-Emissions model:

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build demission "./EM_test_base" -r "{'wflow': 'wflow_piave'}" --fo -vvv

The example above means the following: run **hydromt build** with:

* `demission` : i.e. build a demission model
* `./EM_test_base` : output model folder
* `-r or --region "{'wflow': 'wflow_piave'}"` : derive the model from the existing wflow model located in wflow_piave folder. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options). For DELWAQ, the only possible options is from an existing wflow model.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (grid, segment IDs) have been setup using default parameters. To build a complete model we need the use a configuration file (yml).

### Model setup configuration

The configuration file (yml) contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_configuration_file>`. We have prepared several example files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_delwaq/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html). 

**Almost each DELWAQ model is unique** depending on which substances and sources are included but also what kind of emission data is available. For these reasons, there is no default build configuration file for a demission/delwaq model and still some manual steps are required to build and run a D-Emissions model with HydroMT. You can learn more about these steps in the [docs(hydromt_D-Emissions)](https://deltares.github.io/hydromt_delwaq/latest/advanced/generic_delwaq.html#emissions-modelling-with-d-emission).

We will load an example demission build configuration file for inspection:

In [ ]:
fn_ini = 'delwaq_build_EM.yml'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

Some explanations for the sections the ini file above:

* **setup_basemaps**: prepares the first delwaq model layers (grid, segment ID) and for EM also the geometry.
* **setup_monitoring**: prepares no monitoring points and monitoring areas by compartments (emissions only here).
* **setup_hydrology_forcing**: prepares hydrological data from wflow outputs. Here we skip this step by putting *hydro_forcing_fn=None*.
* **setup_emission_raster**: prepares emissions grid from a raster file. You can repeat this section several times for several rasters. Here we will prepare population data from the global *ghs_pop_2015* dataset.
* **setup_emission_vector**: prepares emissions grid from a vector file. You can repeat this section several times for several rasters. Here we will prepare GDP data from the global *gdp_world* data.

For more information on all the options used (resampling method, NaN values handling...), please check the [docs(model_components)](https://deltares.github.io/hydromt_delwaq/latest/user_guide/components.html)

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets)
This example data is a based on the data which are available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using `-d deltares_data`. 

More background how to write a data catalog yml file can be found in the [hydromt core docs](https://deltares.github.io/hydromt/latest/user_guide/data.html) 

### Setup complete D-Emissions model 

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build demission "./EM_test_full" -r "{'wflow': 'wflow_piave'}" -i delwaq_build_EM.yml -d artifact_data --fo -vv

With this example we build a complete demission model including emission data. Compared to the previous **hydromt build** we have added:
* `-i delwaq_build_EM.yml` : setup configuration file including all components to build and their arguments.

Next we check which files and folders have been created:

* **config**: basic configuration setups for the Delwaq input file .inp. Names of the ASCII files start with the corresponding block in the .inp file.
* **dynamicdata**: Delwaq time-dependant data (hydrological fluxes) in binary format.
* **fews**: additional files for connection with Delft-FEWS (not implemented yet)
* **hydromodel**: data extracted to connect the hydrological model (wflow) and Delwaq (tif format). These files are not mandatory for DELWAQ but produced extra from HydroMT.
* **staticdata**: static (spatial but non time-dependant) data for Delwaq (typically emission data) in binary format. A copy of the data in NetCDF format is also available for easier visualization.
* **geoms**: related geometry files (geojson format). These files are not mandatory for DELWAQ but produced extra from HydroMT.

In [ ]:
import os
root = 'EM_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

You can see that the dynamicdata folder (hydrological input) does not contain any file yet. We will add them in the **update forcing** example notebook.

### Visualize and/or inspect model schematization

* The **delwaq plot** example notebook contains scripts to visualize your model (staticdata, dynamicdata, hydromodel and geoms).

In the meantime, feel free to have a look at some configuration files that were prepared by HydroMT. You can change the file name from the example code below.

In [ ]:
import os
model_path = './EM_test_full'
fn_config = 'config/B3_nrofseg.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)

In the config folder, you can also see three geometry files:

* B7_geometry-parameters.inc (Headers for the binary version)
* B7_geometry.bin (Binary version)
* B7_geometry.inc (ASCII version)

These files are specific for the D-Emissions plugin and specifies geometry properties of the D-Emissions segments.
These properties are:

* Segment area (m2)
* Fraction of the segment area that is paved
* Fraction of the segment area that is unpaved
* Fraction of the segment area that is open water

In [ ]:
import os
model_path = './EM_test_full'
fn_config = 'config/B7_geometry-parameters.inc'
with open(os.path.join(model_path,fn_config), 'r') as f:
    txt = f.read()
print(txt)